### Example of getting photometry.

As an example, to grab the photometry of your favorite, say, LRG targets observed in SV3, you would do something like the following (which we should of course put into a notebook):

In [1]:
import os
import numpy as np
import fitsio
from glob import glob
from astropy.table import Table, vstack

In [5]:
import healpy as hp
hp.nside2resol(4, arcmin=True) / 60

14.658075357087599

In [2]:
zcatdir = '/global/cfs/cdirs/desi/spectro/redux/fuji/zcatalog'
photdir = '/global/cscratch1/sd/ioannis/photocatalog/fuji/tractorphot'

In [ ]:
def get_tractorphot():
    from desimodel.footprint import radec2pix
    from desitarget.sv3.sv3_targetmask import desi_mask

    zcat = Table(fitsio.read(os.path.join(zcatdir, 'ztile-sv3-dark-cumulative.fits'), 'ZCATALOG'))
    I = (zcat['SV3_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * (zcat['ZWARN'] == 0) * (zcat['Z'] > 0.7) * (zcat['Z'] < 0.701)
    zcat = zcat[I]
    print('Selected {} objects'.format(len(zcat)))

    hdr = fitsio.read_header(glob(os.path.join(photdir, 'tractorphot-nside4-hp???-fuji.fits'))[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat['TARGET_RA'], zcat['TARGET_DEC'])
    phot = []
    for pixel in set(pixels):
        J = pixel == pixels
        photfile = os.path.join(photdir, 'tractorphot-nside4-hp{:03d}-fuji.fits'.format(pixel))
        targetids = fitsio.read(photfile, columns='TARGETID')
        K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]
        print('Reading photometry for {} objects from {}'.format(len(K), photfile))
        _phot = fitsio.read(photfile, rows=K)
        phot.append(Table(_phot))
    phot = vstack(phot)
    srt = np.hstack([np.where(tid == phot['TARGETID'])[0] for tid in zcat['TARGETID']]) # better way to sort here??
    phot = phot[srt]
    return zcat, phot

>>> zcat, phot = get_tractorphot()